In [11]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

In [21]:
file_path_list = {}
file_path_list['AVE'] = {}
file_path_list['CTA'] = {}
file_path_list['DC'] = {}
file_path_list['DI'] = {}
file_path_list['ER'] = {}
file_path_list['RE'] = {}
file_path_list['SM'] = {}

In [27]:
file_path_list['AVE']['oa_mine'] = '/data/home/wangys/MELD/dataset/AVE/oa_mine_train_small.json'
file_path_list['CTA']['SimTab'] = '/data/home/wangys/MELD/dataset/CTA/SimTab_Train_few.json'
# file_path_list['CTA']['WebTable'] = '/data/home/wangys/MELD/dataset/CTA/webtable_train_few.json' ## selected-file
file_path_list['CTA']['WebTable'] = '/data/home/wangys/DataSelection-IF/train/CTA/WebTable/WebTable-train.json'

file_path_list['DC']['hospital'] = '/data/home/wangys/MELD/dataset/DC/hospital-train-MoE.json'
file_path_list['DC']['rayyan'] = '/data/home/wangys/MELD/dataset/DC/rayyan-train-MoE.json'
file_path_list['DC']['beer'] = '/data/home/wangys/MELD/dataset/DC/beer-train-MoE.json'
# file_path_list['RE']['RE'] = '/data/home/wangys/MELD/dataset/RE/RE-train.json' ## selected-file
file_path_list['RE']['RE'] = '/data/home/wangys/DataSelection-IF/train/RE/RE-train.json'

In [23]:
file_path_list['DI']['amazon'] = '/data/home/wangys/MELD/dataset/DI/amazon_train_output_wide.json'
file_path_list['DI']['walmart'] = '/data/home/wangys/MELD/dataset/DI/walmart_train_output_wide.json'
file_path_list['DI']['restaurant'] = '/data/home/wangys/MELD/dataset/DI/restaurant_train_output_wide.json'

In [24]:
file_path_list['ER']['semi-text-w'] = '/data/home/wangys/MELD/dataset/ER/semi-text-w-train-MoE.json'
file_path_list['ER']['semi-text-c'] = '/data/home/wangys/MELD/dataset/ER/semi-text-c-train-MoE.json'
file_path_list['ER']['amazon-google'] = '/data/home/wangys/MELD/dataset/ER/amazon-google-train.json'
file_path_list['ER']['walmart-amazon'] = '/data/home/wangys/MELD/dataset/ER/walmart_amazon_train_output.json'
file_path_list['ER']['wdc'] = '/data/home/wangys/MELD/dataset/ER/wdc_all_train_output.json'
file_path_list['ER']['abt-buy'] = '/data/home/wangys/MELD/dataset/ER/ant_buy_train_output.json'
file_path_list['SM']['CMS'] = '/data/home/wangys/MELD/dataset/SM/CMS_train_few_output.json'
file_path_list['SM']['synthea'] = '/data/home/wangys/MELD/dataset/SM/synthea_train_few_output.json'

In [ ]:
df_all = pd.DataFrame()
for task in tqdm(file_path_list):
    for dataset in file_path_list[task].keys():
        raw_file = pd.read_json(file_path_list[task][dataset])
        # raw_file
        ppl_array = np.zeros(len(raw_file))
        for process_num in range(1,9,1): ## maximum of k process
            if os.path.exists('../ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
                ppl_df = pd.read_csv('../ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
                for index,row in ppl_df.iterrows():
                    ppl_array[index] = row[0]
        if np.sum(ppl_array)>10: ## 存在Ppl结果
            top_k = np.argsort(ppl_array)[::-1][:200]
            df_select = raw_file.iloc[top_k].reset_index(drop=True)
        else:
            print('ppl do not exist for {}/{}'.format(task,dataset))
            df_select = raw_file.sample(n=200,random_state=42).reset_index(drop=True)
        df_select = df_select.iloc[:,:3]
        df_select.columns = ['instruction','input','output']
        df_select['task'] = task
        df_select['dataset'] = dataset
        df_all = pd.concat([df_all,df_select])

In [42]:
df_all.to_csv('../train/router/train.csv')

## Expert List for Query

In [43]:
expert_path_list = {}
expert_path_list['AVE'] = {}
expert_path_list['CTA'] = {}
expert_path_list['DC'] = {}
expert_path_list['DI'] = {}
expert_path_list['ER'] = {}
expert_path_list['RE'] = {}
expert_path_list['SM'] = {}

In [44]:
expert_path_list['AVE']['oa_mine'] = '/data/home/wangys/LLaMA-Factory-main/lora_weight/oa_mine/oa_mine_train_small'
expert_path_list['CTA']['SimTab'] = '/data/home/wangys/MELD/lora_weight/expert/CTA_SimTab_train_init'
expert_path_list['CTA']['WebTable'] = '/data/home/wangys/MELD/lora_weight/expert/CTA_WebTable_train_init'

expert_path_list['DC']['hospital'] = '/data/home/wangys/MELD/lora_weight/expert/hospital_train-MoE-Add'
expert_path_list['DC']['rayyan'] = '/data/home/wangys/MELD/lora_weight/expert/rayyan_train-MoE-Add'
expert_path_list['DC']['beer'] = '/data/home/wangys/MELD/lora_weight/expert/beer_train-MoE-Add'
expert_path_list['RE']['RE'] = '/data/home/wangys/MELD/lora_weight/expert/RE-MoE-Add'

In [45]:
expert_path_list['DI']['amazon'] = '/data/home/wangys/MELD/lora_weight/expert/amazon_train-MoE-Add'
expert_path_list['DI']['walmart'] = '/data/home/wangys/MELD/lora_weight/expert/walmart_train-MoE-Add'
expert_path_list['DI']['restaurant'] = '/data/home/wangys/MELD/lora_weight/expert/restaurant_train-MoE-Add'

In [47]:
expert_path_list['ER']['semi-text-w'] = '/data/home/wangys/MELD/lora_weight/expert/semi_text_w-MoE-Add'
expert_path_list['ER']['semi-text-c'] = '/data/home/wangys/MELD/lora_weight/expert/semi_text_c-MoE-Add'
expert_path_list['ER']['amazon-google'] = '/data/home/wangys/MELD/lora_weight/expert/amazon_google-MoE-Add'
expert_path_list['ER']['walmart-amazon'] = '/data/home/wangys/MELD/lora_weight/expert/walmart_amazon-MoE-Add'
expert_path_list['ER']['wdc'] = '/data/home/wangys/MELD/lora_weight/expert/wdc_all-MoE-Add'
expert_path_list['ER']['abt-buy'] = '/data/home/wangys/MELD/lora_weight/expert/ant_buy-MoE-Add'
expert_path_list['SM']['CMS'] = '/data/home/wangys/MELD/lora_weight/expert/CMS_train-MoE-Add'
expert_path_list['SM']['synthea'] = '/data/home/wangys/MELD/lora_weight/expert/synthea_train-MoE-Add'